In [11]:
# import necessary dependencies
import pandas as pd
import numpy as np

df = pd.read_csv("./exams.csv");


In [29]:
# create another column that sums all of the scores
df["total"] = df["math score"] + df["writing score"] + df["reading score"]

rename_columns = {
    "race/ethnicity": "race_ethnicity",
    "parental level of education": "parent_education",
    "test preparation course": "test_prep_course",
    "math score": "math_score",
    "reading score": "reading_score",
    "writing score": "writing_score"
}

df.rename(columns = rename_columns, inplace=True)

df.head()

,gender,race_ethnicity,parent_education,lunch,test_prep_course,math_score,reading_score,writing_score,total
0,male,group A,high school,standard,completed,67,67,63,197
1,female,group D,some high school,free/reduced,none,40,59,55,154
2,male,group E,some college,free/reduced,none,59,60,50,169
3,male,group B,high school,standard,none,77,78,68,223
4,male,group E,associate's degree,standard,completed,78,73,68,219


In [38]:
def generate_mapping(arr):
    dict = {}

    for i in range(len(arr)):
        dict[arr[i]] = i

    return dict

mappings = []
# generate map for non integer columns into dicrete values to calculate correlation between the values
genders = sorted(df.gender.unique())
gender_mapping = generate_mapping(genders)
mappings.append(gender_mapping)
print(gender_mapping)

race_ethnicity = sorted(df.race_ethnicity.unique())
race_ethnicity_mapping = generate_mapping(race_ethnicity)
mappings.append(race_ethnicity_mapping)
print(race_ethnicity_mapping)

parent_education = sorted(df.parent_education.unique())
parent_education_mapping = generate_mapping(parent_education)
mappings.append(parent_education_mapping)
print(parent_education_mapping)

lunch = sorted(df.lunch.unique())
lunch_mapping = generate_mapping(lunch)
mappings.append(lunch_mapping)
print(lunch_mapping)

test_prep_course = sorted(df.test_prep_course.unique())
test_prep_course_mapping = generate_mapping(test_prep_course)
mappings.append(test_prep_course_mapping)
print(test_prep_course_mapping)

{'female': 0, 'male': 1}
{'group A': 0, 'group B': 1, 'group C': 2, 'group D': 3, 'group E': 4}
{"associate's degree": 0, "bachelor's degree": 1, 'high school': 2, "master's degree": 3, 'some college': 4, 'some high school': 5}
{'free/reduced': 0, 'standard': 1}
{'completed': 0, 'none': 1}


In [44]:
# apply mappings to necessary columns
list = df.columns[:5].tolist()

for i in range(len(list)):
    df[list[i]] = df[list[i]].map(mappings[i])

df.head()

,gender,race_ethnicity,parent_education,lunch,test_prep_course,math_score,reading_score,writing_score,total
0,1,0,2,1,0,67,67,63,197
1,0,3,5,0,1,40,59,55,154
2,1,4,4,0,1,59,60,50,169
3,1,1,2,1,1,77,78,68,223
4,1,4,0,1,0,78,73,68,219


In [48]:
# check the correlation between the columns and sort by the absolute(correlation) for the total with other columns
df.corr().sort_values(by=['total'], key= lambda n : np.abs(n))

,gender,race_ethnicity,parent_education,lunch,test_prep_course,math_score,reading_score,writing_score,total
gender,1.000000,0.063231,-0.078676,-0.033962,-0.007653,0.200863,-0.189389,-0.246089,-0.081218
race_ethnicity,0.063231,1.000000,-0.011034,-0.033335,0.029828,0.213156,0.131459,0.133955,0.167890
parent_education,-0.078676,-0.011034,1.000000,-0.033616,0.008912,-0.187776,-0.148998,-0.161748,-0.174806
test_prep_course,-0.007653,0.029828,0.008912,-0.029267,1.000000,-0.151704,-0.245144,-0.315601,-0.249711
lunch,-0.033962,-0.033335,-0.033616,1.000000,-0.029267,0.374431,0.288282,0.319191,0.344386
math_score,0.200863,0.213156,-0.187776,0.374431,-0.151704,1.000000,0.819398,0.805944,0.919834
writing_score,-0.246089,0.133955,-0.161748,0.319191,-0.315601,0.805944,0.954274,1.000000,0.966367
reading_score,-0.189389,0.131459,-0.148998,0.288282,-0.245144,0.819398,1.000000,0.954274,0.970219
total,-0.081218,0.167890,-0.174806,0.344386,-0.249711,0.919834,0.970219,0.966367,1.000000


Since gender has the least significant correlation with the total score <br>
and math_score, writing_score, and reading_score has high positive correlation with total <br>
I will use the column total along with race_ethinicity, parent_education, test_prep_course, and lunch to <br>
find the associative pattern <br>

In [70]:
filtered = df[["race_ethnicity", "parent_education", "lunch", "test_prep_course", "total"]]

filtered.corr().sort_values(by=['total'], key= lambda n : np.abs(n))

,race_ethnicity,parent_education,lunch,test_prep_course,total
race_ethnicity,1.000000,-0.011034,-0.033335,0.029828,0.167890
parent_education,-0.011034,1.000000,-0.033616,0.008912,-0.174806
test_prep_course,0.029828,0.008912,-0.029267,1.000000,-0.249711
lunch,-0.033335,-0.033616,1.000000,-0.029267,0.344386
total,0.167890,-0.174806,0.344386,-0.249711,1.000000


In [172]:
# trying to discretize the total column since it has 194 different values

dict = df.total.value_counts()
dict = dict.sort_index()

keys = dict.keys()


def discretize(dict, keys):
    total = len(dict)
    # threshold to stop merging the columns
    threshold = 0.30
    values = keys
    entropy = []
    entropy.append(calc_entropy(values, dict, total))
    deltas = []
    delta_changes = []

    curr_round = 0
    delta_change = 0
    delta_change_percentage = 0

    while delta_change_percentage < threshold:
        merge(values, dict)
        values = dict.sort_index().keys()

        entropy.append(calc_entropy(values, dict, total))

        if curr_round > 0:
            delta = entropy[curr_round - 1] - entropy[curr_round]
            deltas.append(delta)

        if curr_round > 1:
            temp_delta_change = deltas[curr_round - 2] - deltas[curr_round - 1]
            delta_changes.append(temp_delta_change)

        if curr_round > 2:
            delta_change_percentage = (
                delta_changes[curr_round - 3] - delta_changes[curr_round - 2]
            ) / delta_changes[curr_round - 3]

        curr_round += 1

    return values


# calculating shannon entropy
def calc_entropy(values, dict, total):
    total_entropy = 0

    for i in range(len(values)):
        prob = dict[values[i]] / total

        total_entropy += prob * np.log(1 / prob)

    return total_entropy


def merge(keys, dict):
    diff_arr = []
    curr_min = 1e9
    min_idx = 0
    diff = 0

    for i in range(len(keys) - 1):
        val1, val2 = keys[i], keys[i + 1]

        diff = val2 - val1

        diff_arr.append(diff)
        if diff < curr_min:
            curr_min = diff
            min_idx = i

    val1, val2 = keys[min_idx], keys[min_idx + 1]
    weight1, weight2 = dict[val1], dict[val2]
    total_count = weight1 + weight2

    # using weighted calculation
    merged_key = (weight1 * val1 + weight2 * val2) / total_count
    dict.pop(val1)
    dict.pop(val2)

    dict[merged_key] = total_count

    return


discretized_arr = discretize(dict, keys)
print(discretized_arr)

Float64Index([             65.0,              86.0, 88.33333333333333,
              94.66666666666667,              96.5,              98.0,
                          100.0,             102.5,             108.5,
                          110.0,
              ...
                          286.0,             287.0,             289.0,
                          290.0,             291.0,             293.0,
                          294.0,             295.0,             297.0,
                          300.0],
             dtype='float64', length=189)


Using the entropy discretization, it only reduces to 189 separate bins, which would result in <br>
classification of the "total" column into 191 different values <br>
Thus, no point in trying to discretize more to reduce the differences <br>
Continue to figure out the association pattern